In [20]:
!pip show azureml-train-automl


In [7]:
import azureml.core
from azureml.core import Workspace

# Load the workspace from the saved config file
ws = Workspace.from_config()
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))

Ready to use Azure ML 1.31.0 to work with jp-cpet-eastus-research-ml


In [18]:
from azureml.core import Dataset

default_ds = ws.get_default_datastore()
# Split the dataset into training and validation subsets
cpet_ds = ws.datasets.get("diabetes-unfolded-data")
train_ds, test_ds = cpet_ds.random_split(percentage=0.7, seed=123)
print("Data ready!")

Data ready!


In [19]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "jjp581"

try:
    # Check for existing compute target
    training_cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    # If it doesn't already exist, create it
    try:
        compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS11_V2', max_nodes=2)
        training_cluster = ComputeTarget.create(ws, cluster_name, compute_config)
        training_cluster.wait_for_completion(show_output=True)
    except Exception as ex:
        print(ex)

Found existing cluster, use it.


In [11]:

import azureml.train.automl.utilities as automl_utils

for metric in automl_utils.get_primary_metrics('classification'):
    print(metric)

average_precision_score_weighted
norm_macro_recall
accuracy
precision_score_weighted
AUC_weighted


In [16]:
from azureml.train.automl import AutoMLConfig

automl_config = AutoMLConfig(name='Automated ML Experiment Cardiac',
                             task='classification',
                             compute_target=training_cluster,
                             training_data = cpet_ds,
                             #validation_data = test_ds,
                             label_column_name='NoReadmission',
                             iterations=50,
                             primary_metric = 'norm_macro_recall',
                             n_cross_validations = 5,
                             max_concurrent_iterations=10,
                             featurization='auto',
                             enable_voting_ensemble = False,
                             enable_stack_ensemble = False,
                             model_explainability=True
                             )

print("Ready for Auto ML run.")

Ready for Auto ML run.


In [17]:
from azureml.core.experiment import Experiment
from azureml.widgets import RunDetails

print('Submitting Auto ML experiment...')
automl_experiment = Experiment(ws, 'diabetes_experiment_unfolded')
automl_run = automl_experiment.submit(automl_config)
RunDetails(automl_run).show()
automl_run.wait_for_completion(show_output=True)

Submitting Auto ML experiment...
Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
diabetes_experiment_unfolded,AutoML_22de6768-8e7a-44f1-ad67-c4e7aa09fecb,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

Experiment,Id,Type,Status,Details Page,Docs Page
diabetes_experiment_unfolded,AutoML_22de6768-8e7a-44f1-ad67-c4e7aa09fecb,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.m

{'runId': 'AutoML_22de6768-8e7a-44f1-ad67-c4e7aa09fecb',
 'target': 'jjp581',
 'status': 'Completed',
 'startTimeUtc': '2021-08-09T00:12:50.792229Z',
 'endTimeUtc': '2021-08-09T00:33:15.966645Z',
 'properties': {'num_iterations': '50',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'norm_macro_recall',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '5',
  'target': 'jjp581',
  'AMLSettingsJsonString': '{"path":null,"name":"Automated ML Experiment Cardiac","subscription_id":"2ea3a7ac-8926-4694-9e2c-242e69548c78","resource_group":"jp-cpet-research","workspace_name":"jp-cpet-eastus-research-ml","region":"eastus","compute_target":"jjp581","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"enable_batch_run":false,"enable_run_restructure":false,"start_auxiliary_runs_before_parent_complete":false,"enable_code_generation":false,"iterations":50,"primary_metric":"norm_macro_rec

In [4]:
from azureml.core import Dataset

default_ds = ws.get_default_datastore()
# Split the dataset into training and validation subsets
cpet_ds = ws.datasets.get("diabetes_processed_numeric")
train_ds, test_ds = cpet_ds.random_split(percentage=0.7, seed=123)
print("Data ready!")

Data ready!


In [5]:
from azureml.train.automl import AutoMLConfig

automl_config = AutoMLConfig(name='Automated ML Experiment Diabetes',
                             task='classification',
                             compute_target=training_cluster,
                             training_data = cpet_ds,
                             #validation_data = test_ds,
                             label_column_name='num_readmitted',
                             iterations=50,
                             primary_metric = 'accuracy',
                             n_cross_validations = 4,
                             max_concurrent_iterations=10,
                             featurization='auto',
                             enable_voting_ensemble = False,
                             enable_stack_ensemble = False,
                             model_explainability=True
                             )

print("Ready for Auto ML run.")

Ready for Auto ML run.


In [6]:
from azureml.core.experiment import Experiment
from azureml.widgets import RunDetails

print('Submitting Auto ML experiment...')
automl_experiment = Experiment(ws, 'diabetes_experiment')
automl_run = automl_experiment.submit(automl_config)
RunDetails(automl_run).show()
automl_run.wait_for_completion(show_output=True)

Submitting Auto ML experiment...
Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
diabetes_experiment,AutoML_2febeaea-4653-4290-be79-b9d6f8316157,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

Experiment,Id,Type,Status,Details Page,Docs Page
diabetes_experiment,AutoML_2febeaea-4653-4290-be79-b9d6f8316157,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.m

{'runId': 'AutoML_2febeaea-4653-4290-be79-b9d6f8316157',
 'target': 'jjp581',
 'status': 'Completed',
 'startTimeUtc': '2021-08-07T19:02:22.551356Z',
 'endTimeUtc': '2021-08-07T19:20:29.771602Z',
 'properties': {'num_iterations': '50',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '4',
  'target': 'jjp581',
  'AMLSettingsJsonString': '{"path":null,"name":"Automated ML Experiment Diabetes","subscription_id":"2ea3a7ac-8926-4694-9e2c-242e69548c78","resource_group":"jp-cpet-research","workspace_name":"jp-cpet-eastus-research-ml","region":"eastus","compute_target":"jjp581","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"enable_batch_run":false,"enable_run_restructure":false,"start_auxiliary_runs_before_parent_complete":false,"enable_code_generation":false,"iterations":50,"primary_metric":"accuracy","task_type":

In [6]:
from azureml.train.automl import AutoMLConfig

automl_config = AutoMLConfig(name='Automated ML Experiment Cardiac',
                             task='classification',
                             compute_target=training_cluster,
                             training_data = train_ds,
                             validation_data = test_ds,
                             label_column_name='CardiacLim',
                             iterations=50,
                             primary_metric = 'norm_macro_recall',
                             max_concurrent_iterations=10,
                             featurization='auto',
                             enable_voting_ensemble = False,
                             enable_stack_ensemble = False,
                             allowed_models = ['LogisticRegression','DecisionTree','KNN'],
                             model_explainability = True
                             )

print("Ready for Auto ML run.")

Ready for Auto ML run.


In [7]:
from azureml.core.experiment import Experiment
from azureml.widgets import RunDetails

print('Submitting Auto ML experiment...')
automl_experiment = Experiment(ws, 'cpet-automl-cardiac-interpretable')
automl_run = automl_experiment.submit(automl_config)
RunDetails(automl_run).show()
automl_run.wait_for_completion(show_output=True)

Submitting Auto ML experiment...
Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
cpet-automl-cardiac-interpretable,AutoML_9584958c-0b70-40f3-9ed3-7c89196d8bf4,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

Experiment,Id,Type,Status,Details Page,Docs Page
cpet-automl-cardiac-interpretable,AutoML_9584958c-0b70-40f3-9ed3-7c89196d8bf4,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

****************************************************************************************************

TYPE:         High cardinality feature detection
STATUS

{'runId': 'AutoML_9584958c-0b70-40f3-9ed3-7c89196d8bf4',
 'target': 'jjp581',
 'status': 'Completed',
 'startTimeUtc': '2021-07-06T15:47:23.879524Z',
 'endTimeUtc': '2021-07-06T16:12:47.057552Z',
 'properties': {'num_iterations': '50',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'norm_macro_recall',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': None,
  'target': 'jjp581',
  'AMLSettingsJsonString': '{"path":null,"name":"Automated ML Experiment Cardiac","subscription_id":"2ea3a7ac-8926-4694-9e2c-242e69548c78","resource_group":"jp-cpet-research","workspace_name":"jp-cpet-eastus-research-ml","region":"eastus","compute_target":"jjp581","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"enable_batch_run":false,"enable_run_restructure":false,"start_auxiliary_runs_before_parent_complete":false,"enable_code_generation":false,"iterations":50,"primary_metric":"norm_macro_re

Performing interactive authentication. Please follow the instructions on the terminal.
Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code ASEK8SU4A to authenticate.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code AHRBZMUYM to authenticate.
Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code EL2WJGS77 to authenticate.
Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code AB9NC7PFM to authenticate.
Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://mi